In [35]:
import os
import re
import json
import requests
import datetime
import subprocess

#默认参数
default_config = {
    "channelid_youtube": {
        "YouTube": {
            "update_size": 1,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "720",
            "last_size": 1,
            "media": "m4a"
        },
        "科学火箭叔": {
            "update_size": 15,
            "id": "UC8dD8v_uIA7aRzpqaCoIHKg",
            "title": "科学火箭叔",
            "quality": "1080",
            "last_size": 20,
            "media": "mp4"
        }
    }
}
expected_channelid = {
    "update_size": 1,
    "id": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "YouTube",
    "quality": "720",
    "last_size": 1,
    "media": "m4a"
}

In [36]:
#日志模块
def write_log(log):
    # 获取当前的具体时间
    current_time = datetime.datetime.now()
    # 格式化输出，只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    # 打开文件，并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""
    # 将新的日志内容添加在原有内容之前
    new_contents = formatted_time + " " + log + "\n" + contents
    # 将新的日志内容写入文件
    with open("log.txt", "w") as file:
        file.write(new_contents)
    # 打印当前的具体时间（精确到秒）
    print(formatted_time, log)

In [37]:
# 安装库模块
def library_install(library):
    # 检查库是否已安装
    def is_library_installed():
        try:
            result = subprocess.run([library , '--version'], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 如果库未安装，则尝试安装
    def install_library():
        try:
            result = subprocess.run(['pip', 'install', library], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 如果库已安装，则尝试更新
    def update_library():
        try:
            result = subprocess.run(['pip', 'install', '--upgrade', library], capture_output=True, text=True)
            if result.returncode == 0:
                return True
            else:
                return False
        except FileNotFoundError:
            return False
    # 检查库是否已安装
    if is_library_installed():
        write_log(library + "已安装")
    else:
        write_log(library + "未安装")
    # 如果库已安装，则尝试更新
    if is_library_installed():
        if update_library():
            write_log(library + "更新成功")
        else:
            write_log(library + "更新失败")
    else:  # 如果库未安装，则尝试安装
        if install_library():
            write_log(library + "安装成功")
        else:
            write_log(library + "安装失败")

In [38]:
# 安装/更新yt-dlp
library_install("yt-dlp")

2023-08-07 02:42:42 yt-dlp已安装
2023-08-07 02:42:45 yt-dlp更新成功


In [39]:
# 检查当前文件夹中是否存在config.json文件
if not os.path.exists('config.json'):
    # 如果文件不存在，创建并写入默认字典
    with open('config.json', 'w') as file:
        json.dump(default_config, file, indent=4)
    write_log("不存在配置文件，已新建，默认频道")
    config = default_config
else:
    # 如果文件存在，读取字典并保存到config变量中
    with open('config.json', 'r') as file:
        config = json.load(file)
    write_log("已读取配置文件")

2023-08-07 02:42:45 不存在配置文件，已新建，默认频道


In [40]:
# 从配置文件中获取YouTube的频道
channelid_youtube = config.get("channelid_youtube")
if channelid_youtube is not None:
    write_log("已读取youtube频道信息")
else:
    write_log("youtube频道信息不存在")
# 从配置文件中获取bilibili的频道
channelid_bilibili = config.get("channelid_bilibili")
if channelid_bilibili is not None:
    write_log("已读取bilibili频道信息")
else:
    write_log("bilibili频道信息不存在")

2023-08-07 02:42:45 已读取youtube频道信息
2023-08-07 02:42:45 bilibili频道信息不存在


In [41]:
# 构建文件夹channel_id
folder_path_channel_ids = os.path.join(os.getcwd(), "channel_id")
if not os.path.exists(folder_path_channel_ids):  # 判断文件夹是否存在
    os.makedirs(folder_path_channel_ids)  # 创建文件夹
    write_log("文件夹channel_id创建成功")
else:
    write_log("文件夹channel_id已存在")

2023-08-07 02:42:45 文件夹channel_id已存在


In [42]:
# 读取youtube频道的id
if channelid_youtube is not None:
    channelid_youtube_ids = dict({channel["id"]: channel["title"] for channel in channelid_youtube.values()})
    write_log("读取youtube频道的id成功")
else:
    channelid_youtube_ids = None
# 读取bilibili频道的id
if channelid_bilibili is not None:
    channelid_bilibili_ids = [channelid_bilibili[key]['id'] for key in channelid_bilibili]
    write_log("读取bilibili频道的id成功")
else:
    channelid_bilibili_ids = None

2023-08-07 02:42:45 读取youtube频道的id成功


In [43]:
# 更新Youtube频道xml
for key, value in channelid_youtube_ids.items():
    # 构建 URL
    url = f"https://www.youtube.com/feeds/videos.xml?channel_id={key}"
    # 发送请求并获取响应内容
    response = requests.get(url)
    content = response.text
    # 判断频道id是否正确
    pattern_youtube404 = r"Error 404"  # 设置要匹配的正则表达式模式
    if not re.search(pattern_youtube404, content):
        # 构建文件路径
        file_path = os.path.join(folder_path_channel_ids, f"{key}.txt")
        # 构建文件路径
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
            write_log(f"YouTube频道 {value} 已更新并保存")
        # 构建频道文件夹
        folder_path_channel_id = os.path.join(os.getcwd(), key)
        if not os.path.exists(folder_path_channel_id):  # 判断文件夹是否存在
            os.makedirs(folder_path_channel_id)  # 创建文件夹
            write_log(f"文件夹{key}创建成功")
        else:
            write_log(f"文件夹{key}已存在")
    else:
        write_log(f"YouTube频道 {value} ID不正确无法获取")
        del channelid_youtube_ids[key]  # 删除错误ID

2023-08-07 02:42:45 YouTube频道 YouTube 已更新并保存
2023-08-07 02:42:45 文件夹UCBR8-60-B28hp2BmDPdntcQ已存在


2023-08-07 02:42:45 YouTube频道 科学火箭叔 已更新并保存
2023-08-07 02:42:45 文件夹UC8dD8v_uIA7aRzpqaCoIHKg已存在


In [44]:
print(channelid_youtube_ids)

{'UCBR8-60-B28hp2BmDPdntcQ': 'YouTube', 'UC8dD8v_uIA7aRzpqaCoIHKg': '科学火箭叔'}
